In [ ]:
import numpy as np
import torch
import torch.nn as nn
from torchvision import transforms
from PIL import Image
import torchvision as tv
from IPython.display import display, clear_output
from tqdm import tqdm

# Noisy example
Take an image, and add some noise to it

## Difference operator
The difference operator is like a convolution by a particular kernel.

In [ ]:
def difference_operator():
    pass

# Proximal operators
We can calculate the proximal operators for each of the terms of the sum separately, then use these separate computations in an algorithm such as forward-backward or ADMM.

In [ ]:
def proximal_L2():
    pass

# Treating the finite difference operator

# Lagrangian formulation

We could treat the problem as minimizing

$$\frac{1}{2}\|Ax-b\|^2_2 + \|y\|_1$$
under the constraints $y = Dx$ and $x\in C$

# Deblurring the image

# Several algorithms for optimizing the loss function

### Adam

In [ ]:
def open_image_bw(path, downsample=1):
    transform = transforms.Compose([
        transforms.ToTensor(),  # Convert the image to a tensor
    ])
    img = Image.open(path)
    original_width, original_height = img.size
    img = img.convert('L')
    img = img.resize((original_width // downsample, original_height // downsample))
    return transform(img)

In [ ]:
gray_dog_tensor = open_image_bw('dog.jpg', downsample=10)
disc_tensor = open_image_bw('rond.jpg', downsample=2)

In [ ]:
class FiniteDifference(nn.Module):
    def __init__(self):
        super().__init__()
        filter = torch.tensor([
            [-1, 1],
            [0, 0]
        ], dtype=torch.float32).reshape(1, 1, 2, 2)
        filter = filter / torch.norm(filter, p=2)
        self.filter = nn.Parameter(filter, requires_grad=False)
    
    def forward(self, image):
        return torch.nn.functional.conv2d(image, self.filter, padding = 1)

class GaussianBlur(nn.Module):
    def __init__(self, kernel_size, sigma):
        super().__init__()
        self.gaussian_blur = tv.transforms.GaussianBlur(kernel_size, sigma)
    
    def forward(self, image):
        return self.gaussian_blur(image)
    
class ReconstructionLoss(nn.Module):
    def __init__(self, image, penalty_weight, gaussian_std, kernel_size):
        super().__init__()
        self.image = nn.Parameter(image, requires_grad=False)
        self.gaussian_blur = GaussianBlur(kernel_size=kernel_size, sigma=gaussian_std)
        self.finite_difference = FiniteDifference()
        reconstruction = image.clone().detach()
        self.reconstruction = nn.Parameter(reconstruction, requires_grad=True)
        self.penalty_weight = nn.Parameter(torch.tensor(penalty_weight), requires_grad=False)
    def forward(self):
        blurred_reconstruction = self.gaussian_blur(self.reconstruction)
        l2_loss = 1/2 * torch.norm(blurred_reconstruction - self.image, p=2)**2
        finite_difference_reconstruction = self.finite_difference(self.reconstruction)
        l1_loss = torch.norm(finite_difference_reconstruction, p=2)
        return l2_loss + self.penalty_weight*l1_loss

### Add noise

In [ ]:
def add_noise(image, std_noise, gaussian_std=10, kernel_size=51, print_image=True,image_size=(1500,900)):
    # Gaussian blur + noise
    # Show original
    printed_image = image.clone().detach()
    if print_image:
        print(f"original shape {printed_image.shape}")
    printed_image = transforms.ToPILImage()(printed_image).resize(image_size)
    if print_image:
        display(printed_image)
    noise = torch.randn(image.shape) * std_noise
    blurred_image = tv.transforms.functional.gaussian_blur(image, kernel_size=kernel_size, sigma=gaussian_std)
    noisy_image = blurred_image + noise
    noisy_image = torch.clamp(noisy_image, 0, 1)
    printed_image = noisy_image.clone().detach()
    if print_image:
        print(f"new shape {printed_image.shape}")
    printed_image = transforms.ToPILImage()(printed_image).resize(image_size)
    if print_image:
        display(printed_image)
    return noisy_image

In [ ]:
kernel_size = 3
gaussian_std = 0.001
std_noise = 0.03
noisy_image = add_noise(gray_dog_tensor, std_noise=std_noise, gaussian_std=gaussian_std, kernel_size=kernel_size, print_image=True)

In [ ]:
kernel_size = 3
gaussian_std = 0.001
std_noise = 0.03
noisy_image = add_noise(gray_dog_tensor, std_noise=std_noise, gaussian_std=gaussian_std, kernel_size=kernel_size, print_image=False)
print(f"{kernel_size=}, {gaussian_std=}")
reconstruction_module = ReconstructionLoss(noisy_image, penalty_weight=0.5, gaussian_std=gaussian_std, kernel_size=kernel_size)
# Now optimize the reconstruction module
optimizer = torch.optim.AdamW(reconstruction_module.parameters(), lr=0.01)
#reconstruction_module.to('cuda')
pbar = tqdm(range(1000))
for i in pbar:
    optimizer.zero_grad()
    loss = reconstruction_module()
    loss.backward()
    optimizer.step()
    if i % 10 == 0:
        pbar.set_description(f"Loss: {loss.item():.9f}")

    if i % 1000 == 0:
        # Clear the previous output
        clear_output(wait=True)
        print(f"Epoch {i}, Loss: {loss.item()}")
        
        # Clone the tensor, detach it from the computation graph, and move it to CPU
        result = reconstruction_module.reconstruction.clone().detach().cpu()
        # Convert to PIL image
        pil_image = transforms.ToPILImage()(result).resize((1500,900))
        # Display the image in the notebook
        display(pil_image)


In [ ]:
result.min()

# Condat-Vu

On considère le problème d'optimisation:
\begin{equation}
\min_{x \in \mathbb{R^{n \times n} }} \frac{1}{2} \|x-z \|_{2}^{2} + \lambda \| Dx \|_{1} + i_c(x)
\end{equation}
où $C = \{ u \in \mathbb{R}^{n \times n} | \|u-z \|_{2}^{2} \leq \sigma ^2 \} $, $\lambda >0$ et $D$ l'opérateur de différences finies.


Nous allons appliquer l'algorithme de Condat-Vu, avec $f= i_c$,$h = \frac{1}{2} \| \cdot -z \| _{2}^{2}$, $g=\| \cdot \|_{1}$ et $L = D$.


On calcule:

\begin{equation}
prox_{\tau f} (x) = P_{C}(x)
\end{equation}

\begin{equation}
prox_{\gamma g^{*}} (x) = (\varphi (x_i) )_{1 \leq i \leq n^2 }
\end{equation}

\begin{equation}
\nabla{h}(x) = x-z
\end{equation}
où:
\begin{equation}
\begin{array}{l|rcl}
\varphi : & \mathbb{R} & \longrightarrow & \mathbb{R} \\
    & x & \longmapsto & \begin{cases}-1 \text{ , si $x<-1$,} \\ x \text{ , si $|x| \leq 1$,} \\ 1 \text{ , si $x>1$,} \end{cases}
    \end{array}
    \end{equation}

In [ ]:
#calcul des prox de f et g

def prox_f(x,z,sigma):
    if torch.linalg.norm(x-z)>sigma:
        return sigma*(x-z)/torch.linalg.norm(x-z) + z 
    else:
        return x

def varphi(x):
    if x<-1:
        return -1
    if x>1:
        return 1
    else:
        return x
    
def prox_g(x):
    y=torch.zeros(size=x.size())
    for i in range(x.size()[2]):
        for j in range(x.size()[3]):
            y[0,:,i,j]=varphi(x[0,:,i,j])
            y[1,:,i,j]=varphi(x[1,:,i,j])
    return y


In [ ]:
#D et D^T

def GradientHor(x):
    y=x-torch.roll(x,1,dims=2)
    y[0,:,0]=0
    return y

def GradientVer(x):
    y=x-torch.roll(x,1,dims=1)
    y[0,0,:]=0
    return y

def DivHor(x):
    N=(x[0,0]).numel()
    y=x-torch.roll(x,-1,dims=2)
    y[0,:,0]=-x[0,:,1]
    y[0,:,N-1]=x[0,:,N-1]
    return y

def DivVer(x):
    N=len(x[0])
    y=x-torch.roll(x,-1,dims=1)
    y[0,0,:]=-x[0,1,:]
    y[0,N-1,:]=x[0,N-1,:]
    return y

def Psi(x):
    y=[]
    y.append(GradientHor(x).numpy())
    y.append(GradientVer(x).numpy())
    y=np.asarray(y)
    return torch.asarray(y)

def Psit(y):
    x=DivHor(y[0])+DivVer(y[1])
    return x


In [ ]:
#Condat-Vu

def condat_vu(x_0,v_0,z, n_max, tau, gamma, sigma):
    p = []
    q = []
    x = [x_0]
    v = [v_0]
    lambda_ = 1

    for n in range(n_max):
        p.append( prox_f( x[-1] - tau*(x[-1] -z + Psit(v[-1])), z, sigma))
        q.append( prox_g( v[-1] + gamma*( Psi(2*p[-1] - x[-1]) ) ) )
        x_temp,v_temp=x[-1],v[-1]
        x.append(x_temp + lambda_*(p[-1] - x_temp))
        v.append(v_temp + lambda_*(q[-1] - v_temp))
        print(n)
    return x

In [ ]:
kernel_size = 1
gaussian_std = 0.001
std_noise = 0.2
noisy_image_2 = add_noise(gray_dog_tensor, std_noise=std_noise, gaussian_std=gaussian_std, kernel_size=kernel_size, print_image=True)
noisy_image_3 = add_noise(disc_tensor, std_noise=std_noise, gaussian_std=gaussian_std, kernel_size=kernel_size, print_image=True,image_size=(800,800))

In [ ]:
x_0=noisy_image_2.clone().detach()
v_0=Psi(x_0)
z=noisy_image_2.clone().detach()
iteration_2=condat_vu(x_0,v_0,z,n_max=5,tau=0.1,gamma=0.3,sigma=20)

In [ ]:
x_0=noisy_image_3.clone().detach()
v_0=Psi(x_0)
z=noisy_image_3.clone().detach()
iteration_3=condat_vu(x_0,v_0,z,n_max=25,tau=0.1,gamma=0.3,sigma=180)

In [ ]:
image_3= torch.clamp(iteration_3[-1],0,1)
image_3.size()
pimage_3 = transforms.ToPILImage()(image_3).resize((800,800))
# Display the image in the notebook
display(pimage_3)

In [ ]:
image_2= torch.clamp(iteration_2[-1],0,1)
image_2.size()
pimage_2 = transforms.ToPILImage()(image_2).resize((1500,900))
# Display the image in the notebook
display(pimage_2)

# Checking the convergence speed for each of the algorithms

In [ ]:
def accuracy_logger():
    # this function logs the value of the loss function, and the time taken by the iteration
    pass

In [ ]:
# Comparison of convergence. Do a loglog plot of the losses vs. iteration number, and vs. time.

# Testing the results